In [1]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 27.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatib

In [2]:
# Importing Relevant Packages
from ortools.sat.python import cp_model
import pandas as pd
import time
from dataclasses import dataclass
from typing import List
import re
import csv
from collections import defaultdict
import time
import matplotlib.pyplot as plt
import numpy as np
from statistics import stdev
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import math
import random
from itertools import product
import json
from google.colab import files
from io import BytesIO

In [3]:
# Forming the Data Structures
@dataclass
class Patient: # Forming the structure for the patients dataset
    id: int # Unique patient identifier
    name: str # Anonymised patient label
    age: int # Age of patient in years
    gender: str # Gender of patient
    admission_day: int # Day the patient was admitted in the hospital
    release_day: int # Day the patient was released from the hospital
    prefered_capacity: int # How many number of beds it is preffered the room to have
    spec_id: int # ID of the required specialism
    spec_days: int # Number of days a patient requires the specialism
    needs: List[int] # ID of features the patient needs
    prefers: List[int]  # ID of features the patient prefers

@dataclass
class Room: # Forming the structure for the room dataset
    id: int # Unique room identifier
    room_number: int  # Room number in hospital
    department: int # ID of the department the room belongs
    capacity: int # How many beds the room has
    gender: str # Gender the room is designated for
    has: List[int]  # Equipment the room has
    spec_ids: List[int]  # IDs of the specialisms supported by this room
    penalties: List[int] # Penalties of specialisms

In [4]:
uploaded = files.upload()

# Load 'cleaned_rooms1.csv'
rooms_df    = pd.read_csv(BytesIO(uploaded['cleaned_rooms1.csv']))
# Load 'cleaned_patients1.csv'
patients_df = pd.read_csv(BytesIO(uploaded['cleaned_patients1.csv']))

Saving cleaned_patients1.csv to cleaned_patients1.csv
Saving cleaned_rooms1.csv to cleaned_rooms1.csv


In [6]:
# Build hospital data from CSV
def build_hospital_data():
    # Read room and patient data from CSV
    rooms_df = pd.read_csv("cleaned_rooms1.csv")
    patients_df = pd.read_csv("cleaned_patients1.csv")

    # List for Room objects
    rooms = [
        Room(
            id=i, # Unique ID for each room
            room_number=int(row["RoomNumber"]), # Room number
            department=int(row["DepartmentID"]), # Department ID
            capacity=int(row["Capacity"]), # Room capacity
            gender=row["Gender"], # Gender designation
            has=[ # List for equipment the room has
                int(row["Telemetry"]),
                int(row["Oxygen"]),
                int(row["Nitrogen"]),
                int(row["Television"])
            ],
            spec_ids=[ # List for specialism IDs
                int(row["ReqSpecialism1"]),
                int(row["ReqSpecialism2"]),
                int(row["ReqSpecialism3"])
            ],
            penalties=[ # List for specialism penalties
                int(row["PenaltySpecialism1"]),
                int(row["PenaltySpecialism2"]),
                int(row["PenaltySpecialism3"])
            ]
        )
        for i, row in rooms_df.iterrows() # Iterate through each row
    ]

    # List for Patient objects
    patients = [
        Patient(
            id=i, # Unique patient ID
            name=row["Name"], # Patient label
            age=int(row["Age"]), # Patient age
            gender=row["Gender"], # Patient gender
            admission_day=int(row["AdmissionDay"]), # Admission day
            release_day=int(row["ReleaseDay"]), # Release day
            spec_id=int(row["SpecialismID"]), # Specialism ID
            spec_days=int(row["SpecialismDays"]), # Number of days for specialism
            prefered_capacity=int(row["PreferredRoomCapacity"]), # Preferred room capacity
            needs=[ # List for needed equipment
                int(row["NeedsTelemetry"]),
                int(row["NeedsOxygen"]),
                int(row["NeedsNitrogen"]),
                int(row["NeedsTV"])
            ],
            prefers=[ # List of preferred equipment
                int(row["PrefersTelemetry"]),
                int(row["PrefersOxygen"]),
                int(row["PrefersNitrogen"]),
                int(row["PrefersTV"])
            ]
        )
        for i, row in patients_df.iterrows() # Iterate through each row
    ]

    # Return the lists
    return rooms, patients

In [7]:
# Print the final hospital allocation
class HospitalAllocationPrinter(cp_model.CpSolverSolutionCallback):
    # Initialize the callback
    def __init__(self, seats, patients, rooms):
        super().__init__()
        self.__seats = seats # Store boolean variables representing assignments
        self.__patients = patients # Store patient objects
        self.__rooms = rooms # Store room objects
        self.__solution_found = False # Flag when solution has been found

    # Callback when an improving solution is found
    def on_solution_callback(self):
        self.__solution_found = True

    # Print the final solution
    def print_final_solution(self, solver):
        print("\nFinal solution:") # Header
        # Iterate through each room
        for room in self.__rooms:
            # Room details
            print(f"\nRoom {room.id} (RoomNum: {room.room_number}, Dept: {room.department}, Capacity: {room.capacity}, Gender: {room.gender}):")
            assigned = False
            # Iterate through each patient to see if they are assigned to the current room
            for patient in self.__patients:
                if solver.BooleanValue(self.__seats[(room.id, patient.id)]):
                    # Patient details
                    print(f"  - Patient {patient.name} (Age: {patient.age}, Gender: {patient.gender})")
                    assigned = True
            # If no assigned patients print "(empty)"
            if not assigned:
                print("  (empty)")

    # Check if a solution has been found
    def solution_found(self):
        return self.__solution_found

In [8]:
# Compute Gini Coefficient
def compute_gini(x):
    x = np.array(x) # Non-negative values
    if np.all(x == 0):  # If everyone has zero, then it is perfect equitablity
        return 0.0
    diff_sum = np.sum(np.abs(np.subtract.outer(x, x))) # Or else pairwise normalised absolute differences are computed
    return diff_sum / (2 * len(x) * np.sum(x))

# Jain's Index is computed
def compute_jain(x):
    x = np.array(x) # Array with non-negative values
    if np.sum(x) == 0:
        return 1.0 # If everyone has zero, then it is perfect equitablity
    return (np.sum(x)**2) / (len(x) * np.sum(x**2)) # Or else this is returned

In [ ]:
# Compatibility penalty of allocating a patient to a room
def compatibility_penalty(p, r, weights):
    pen = 0 # Initializing penalty to 0

    # Preferred room capacity
    if r.capacity > p.prefered_capacity:
        pen += weights['Wcap']

    # Gender mismatch
    if r.gender in ['F', 'M'] and r.gender != p.gender:
        pen += weights['Wgen'] # Add penalty

    # Preferred equipment unmet
    equip_len = min(len(p.prefers), len(r.has))
    if any(p.prefers[i] == 1 and r.has[i] == 0 for i in range(equip_len)):
        pen += weights['Wpeq'] # Add penalty

    # Needed equipment unmet
    need_len = min(len(p.needs), len(r.has))
    if any(p.needs[i] == 1 and r.has[i] == 0 for i in range(need_len)):
        pen += weights['Wneq'] # Add penalty

    # Department age limits
    if (r.department == 1 and p.age < 65) or (r.department == 4 and p.age > 16):
            pen += weights['Wage'] # Add penalty

    # Specialism
    if p.spec_id is not None:
        if p.spec_id in r.spec_ids:
            idx = r.spec_ids.index(p.spec_id)
            penalty_level = r.penalties[idx] # Get penalty level associated with this specialism
            if penalty_level == 2:
                pen += weights['Wspec_minor']  # Add penalty
            elif penalty_level >= 3:
                pen += weights['Wspec_major']  # Add penalty
        else:
            pen += weights['Wspec_mismatch']  # Add penalty

    return pen # Return total penalty for this allocation

In [18]:
def solve_hospital_allocation(
    Wcap=0, Wgen=0, Wpeq=0, Wneq=0, Wage=0,
    Wspec_minor=0, Wspec_major=0, Wspec_mismatch=0,
    Wfair_high=0, Wfair_med=0, Wfair_low=0,
    Wmove=0
):

    rooms, patients = build_hospital_data()

    weights = {
        'Wcap': Wcap, 'Wgen': Wgen, 'Wpeq': Wpeq, 'Wneq': Wneq, 'Wage': Wage,
        'Wspec_minor': Wspec_minor, 'Wspec_major': Wspec_major, 'Wspec_mismatch': Wspec_mismatch, 'Wfair_high': Wfair_high,
        'Wfair_med': Wfair_med, 'Wfair_low': Wfair_low, 'Wmove': Wmove
    }
    random.seed(42)
    np.random.seed(42)

    total_duration = 0.0
    historical_penalties = {p.id: 0 for p in patients}
    previous_assignments = {}
    daily_assignment_penalties = {}

    last_admission_day = max(p.admission_day for p in patients)
    all_days = range(last_admission_day + 1)

    all_assignments = {}
    results = []

    for current_day in all_days:
        print(f"Solving day {current_day}...")
        prev_penalties = daily_assignment_penalties.get(current_day - 1, {})
        max_prev_penalty = max(prev_penalties.values(), default=None)

        for p in patients:

            if p.admission_day == current_day:
                if random.random() < 0.3:
                    delay = random.randint(1, 2)
                    duration = p.release_day - p.admission_day
                    p.admission_day += delay
                    p.release_day = p.admission_day + duration

                    if not hasattr(p, "arrival_delays"):
                        p.arrival_delays = []
                    p.arrival_delays.append({
                        "day": current_day,
                        "delay": delay,
                        "new_admission": p.admission_day,
                        "new_release": p.release_day,
                    })

            if p.admission_day == current_day:
                if random.random() < 0.3:
                    advance = random.randint(1, 2)
                    duration = p.release_day - p.admission_day
                    p.admission_day = max(0, p.admission_day - advance)
                    p.release_day = p.admission_day + duration

                    if not hasattr(p, "early_arrivals"):
                        p.early_arrivals = []
                    p.early_arrivals.append({
                        "day": current_day,
                        "advance": advance,
                        "new_admission": p.admission_day,
                        "new_release": p.release_day,
                    })

            if p.release_day == current_day:
                if random.random() < 0.3:
                    extension = random.randint(1, 2)
                    p.release_day += extension

                    if not hasattr(p, "late_releases"):
                        p.late_releases = []
                    p.late_releases.append({
                        "day": current_day,
                        "extension": extension,
                        "new_release": p.release_day,
                    })

            if p.release_day == current_day:
                if random.random() < 0.3:
                    reduction = random.randint(1, 2)
                    new_release = max(p.admission_day + 1, p.release_day - reduction)
                    if new_release < p.release_day:
                        if not hasattr(p, "early_releases"):
                            p.early_releases = []
                        p.early_releases.append({
                            "day": current_day,
                            "early_by": p.release_day - new_release,
                            "new_release": new_release,
                        })
                        p.release_day = new_release

        present_patients = [p for p in patients if p.admission_day <= current_day < p.release_day]
        if not present_patients:
            continue

        if current_day > 0:
            all_fairness_scores = []
            for p in present_patients:
                for r in rooms:
                    base_pen = compatibility_penalty(p, r, weights)
                    bias = 1 + historical_penalties[p.id] / max(1, current_day + 1)
                    fairness_score = base_pen * bias
                    all_fairness_scores.append(fairness_score)

            scores_array = np.array(all_fairness_scores)
            threshold_low = np.percentile(scores_array, 25)
            threshold_high = np.percentile(scores_array, 67)

            print(f"Day {current_day} thresholds - Low: {threshold_low:.2f}, High: {threshold_high:.2f}")
        else:
            all_base_scores = []
            for p in present_patients:
                for r in rooms:
                    base_pen = compatibility_penalty(p, r, weights)
                    all_base_scores.append(base_pen)

            scores_array = np.array(all_base_scores)
            threshold_low = np.percentile(scores_array, 25)
            threshold_high = np.percentile(scores_array, 67)

            print(f"Day {current_day} (initial) thresholds - Low: {threshold_low:.2f}, High: {threshold_high:.2f}")

        model = cp_model.CpModel()
        seats = {}
        for r in rooms:
            for p in present_patients:
                seats[(r.id, p.id)] = model.NewBoolVar(f"s_r{r.id}_p{p.id}_d{current_day}")

        for p in present_patients:
            model.Add(sum(seats[(r.id, p.id)] for r in rooms) == 1)

        for r in rooms:
            model.Add(sum(seats[(r.id, p.id)] for p in present_patients) <= r.capacity)

        num_equipment_types = min(
            max(len(r.has) for r in rooms),
            max(len(p.needs) for p in present_patients)
        )

        for i in range(num_equipment_types):
            rooms_with_equipment = [r for r in rooms if len(r.has) > i and r.has[i] == 1]
            total_available = len(rooms_with_equipment)

            patients_with_need = [p for p in present_patients if len(p.needs) > i and p.needs[i] == 1]
            total_needed = len(patients_with_need)

            if total_available >= len(patients_with_need):
                for p in patients_with_need:
                    assign_with_eq = [seats[(r.id, p.id)] for r in rooms_with_equipment]
                    model.AddBoolOr(assign_with_eq)


        viol_pref_cap = []
        viol_gender = []
        viol_pref_eq = []
        viol_need_eq = []
        viol_age = []
        viol_spec_minor = []
        viol_spec_major = []
        viol_spec_mismatch = []
        viol_fairness_high = []
        viol_fairness_medium = []
        viol_fairness_low = []
        move_penalties = []


        for p in present_patients:
            prev_room = previous_assignments.get(p.id)
            for r in rooms:
                s = seats[(r.id, p.id)]

                if r.capacity > p.prefered_capacity:
                    viol_pref_cap.append(s)
                if r.gender in ('M','F') and r.gender != p.gender:
                    viol_gender.append(s)
                equip_len = min(len(p.prefers), len(r.has))
                if any(p.prefers[i] == 1 and r.has[i] == 0 for i in range(equip_len)):
                    viol_pref_eq.append(s)

                equip_need_len = min(len(p.needs), len(r.has))
                if any(p.needs[i] == 1 and r.has[i] == 0 for i in range(equip_need_len)):
                    viol_need_eq.append(s)
                if (r.department==1 and p.age<65) or (r.department==4 and p.age>16):
                    viol_age.append(s)

                if p.spec_id in r.spec_ids:
                    idx = r.spec_ids.index(p.spec_id)
                    pen = r.penalties[idx]
                    if pen == 2:
                        viol_spec_minor.append(s)
                    elif pen >= 3:
                        viol_spec_major.append(s)
                else:
                    viol_spec_mismatch.append(s)

                base_pen = compatibility_penalty(p, r, weights)
                if current_day > 0:
                    bias = 1 + historical_penalties[p.id] / max(1, current_day + 1)
                    fairness_score = base_pen * bias
                else:
                    fairness_score = base_pen

                if fairness_score > threshold_high:
                    viol_fairness_high.append(s)
                elif fairness_score > threshold_low:
                    viol_fairness_medium.append(s)
                elif fairness_score < threshold_low:
                    viol_fairness_low.append(s)
                if prev_room is not None and r.id != prev_room:
                    move_penalties.append(s)


        normalizer = len(present_patients) * len(rooms)

        def normalized_sum(viol_list):
            return sum(viol_list) * (1.0 / normalizer)

        model.Minimize(
              Wcap * normalized_sum(viol_pref_cap)
            + Wgen * normalized_sum(viol_gender)
            + Wpeq * normalized_sum(viol_pref_eq)
            + Wneq * normalized_sum(viol_need_eq)
            + Wage * normalized_sum(viol_age)
            + Wspec_minor * normalized_sum(viol_spec_minor)
            + Wspec_major * normalized_sum(viol_spec_major)
            + Wspec_mismatch * normalized_sum(viol_spec_mismatch)
            + Wfair_low * normalized_sum(viol_fairness_low)
            + Wfair_med * normalized_sum(viol_fairness_medium)
            + Wfair_high * normalized_sum(viol_fairness_high)
            + Wmove * normalized_sum(move_penalties)
        )

        solver = cp_model.CpSolver()
        start = time.time()
        status = solver.Solve(model)
        day_duration = time.time() - start
        total_duration += day_duration

        print(f"Day {current_day} solved in {day_duration:.2f}s — {solver.StatusName(status)}")

        if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
            assign = {}
            violation_scores = []
            daily_score = 0

            for p in present_patients:
                assigned_room = None
                for r in rooms:
                    if solver.Value(seats[(r.id, p.id)]):
                        assigned_room = r
                        assign[p.id] = r.id
                        previous_assignments[p.id] = r.id
                        break

                if assigned_room:
                    pen = compatibility_penalty(p, assigned_room, weights)
                    daily_score += pen
                    historical_penalties[p.id] += pen

                    equipment_labels = ['telemetry', 'oxygen', 'nitrogen', 'television']
                    max_equipment_index = min(len(p.needs), len(assigned_room.has), len(equipment_labels))

                    missing_eq = [
                        i for i in range(max_equipment_index)
                        if p.needs[i] == 1 and assigned_room.has[i] == 0
                    ]

                    equip_len = min(len(p.prefers), len(assigned_room.has))
                    need_len = min(len(p.needs), len(assigned_room.has))
                    spec_minor = 0
                    spec_major = 0
                    spec_mismatch = 0

                    if p.spec_id in assigned_room.spec_ids:
                        idx = assigned_room.spec_ids.index(p.spec_id)
                        spec_pen = assigned_room.penalties[idx]
                        if spec_pen == 2:
                            spec_minor += 1
                        elif spec_pen >= 3:
                            spec_major += 1
                    else:
                        spec_mismatch += 1

                    violation_scores.append({
                        'patient_id': p.id,
                        'room_id': assigned_room.id,
                        'total_score': pen,
                        'historical_penalty': historical_penalties[p.id],
                        'violations': {
                            'capacity': int(assigned_room.capacity > p.prefered_capacity),
                            'gender': int(assigned_room.gender in ['F', 'M'] and assigned_room.gender != p.gender),
                            'equipment_pref': int(any(p.prefers[i] == 1 and assigned_room.has[i] == 0 for i in range(equip_len))),
                            'equipment_need': int(any(p.needs[i] == 1 and assigned_room.has[i] == 0 for i in range(need_len))),
                            'age': int((assigned_room.department == 1 and p.age < 65) or (assigned_room.department == 4 and p.age > 16)),
                            'specialism': int(spec_minor > 0 or spec_major > 0 or spec_mismatch > 0),
                            'moved': int(previous_assignments.get(p.id) is not None and assigned_room.id != previous_assignments.get(p.id)),
                        }
                    })

                all_assignments[current_day] = {
                    'assignments': assign,
                    'violation_scores': violation_scores
                }

                move_count = sum(v['violations']['moved'] for v in violation_scores)


            all_assignments[current_day] = {
                'assignments': assign,
                'violation_scores': violation_scores
            }

            results.append({
                'day': current_day,
                'num_patients': len(present_patients),
                'preferred_capacity_violations': sum(v['violations']['capacity'] for v in violation_scores),
                'gender_violations': sum(v['violations']['gender'] for v in violation_scores),
                'pref_equip_violations': sum(v['violations']['equipment_pref'] for v in violation_scores),
                'need_equip_violations': sum(v['violations']['equipment_need'] for v in violation_scores),
                'age_violations': sum(v['violations']['age'] for v in violation_scores),
                'specialism_violations': sum(v['violations']['specialism'] for v in violation_scores),
                'move_violations': sum(v['violations']['moved'] for v in violation_scores),
                'fairness_violations': sum(1 for v in violation_scores if v['total_score']  > threshold_high),
                'total_violation_score': daily_score,
                'threshold_low': threshold_low,
                'threshold_high': threshold_high
            })

    if results:
        df = pd.DataFrame(results)
        total_days = len(all_days)
        success_rate = len(results) / total_days * 100
        patient_ids = list(historical_penalties.keys())
        stay_length = {p.id: max(1, p.release_day - p.admission_day) for p in patients}
        avg_daily_penalty = {pid: historical_penalties[pid]/stay_length[pid] for pid in stay_length}
        penalty_vals = list(avg_daily_penalty.values())

        std_dev = np.std(penalty_vals)
        gini = compute_gini(penalty_vals)
        jain = compute_jain(penalty_vals)

        avg_penalties = [historical_penalties[pid]/stay_length[pid] for pid in patient_ids]
        max_penalty_idx = np.argmax(penalty_vals)
        min_penalty_idx = np.argmin(penalty_vals)
        total_fairness_penalties = df['fairness_violations'].sum()

        print(f"Total Fairness Penalties: {total_fairness_penalties}")


        summary_stats = {
            "Metric": [
                "Total Solving Time (s)",
                "Success Rate (%)",
                "Fairness - StdDev of Daily Penalty",
                "Fairness - Gini Coefficient",
                "Fairness - Jain Index",
                "Total Penalty - Mean",
                "Total Penalty - StdDev",
                "Total Penalty - Min",
                "Total Penalty - Max",
                "Avg Daily Penalty - Mean",
                "Avg Daily Penalty - StdDev",
                "Avg Daily Penalty - Min",
                "Avg Daily Penalty - Max",
                "Patients with Zero Penalty",
                "Patients with High Penalty (>mean+std)",
                f"Most Penalized Patient (ID {patient_ids[max_penalty_idx]})",
                f"Least Penalized Patient (ID {patient_ids[min_penalty_idx]})",
            ],
            "Value": [
                f"{total_duration:.2f}",
                f"{success_rate:.2f}",
                f"{std_dev:.2f}",
                f"{gini:.2f}",
                f"{jain:.2f}",
                f"{np.mean(penalty_vals):.2f}",
                f"{np.std(penalty_vals):.2f}",
                f"{np.min(penalty_vals):.2f}",
                f"{np.max(penalty_vals):.2f}",
                f"{np.mean(avg_penalties):.2f}",
                f"{np.std(avg_penalties):.2f}",
                f"{np.min(avg_penalties):.2f}",
                f"{np.max(avg_penalties):.2f}",
                f"{sum(1 for p in penalty_vals if p == 0)}",
                f"{sum(1 for p in penalty_vals if p > np.mean(penalty_vals) + np.std(penalty_vals))}",
                f"Total: {penalty_vals[max_penalty_idx]:.2f}, Avg: {avg_penalties[max_penalty_idx]:.2f}",
                f"Total: {penalty_vals[min_penalty_idx]:.2f}, Avg: {avg_penalties[min_penalty_idx]:.2f}",
            ]
        }

        summary_df = pd.DataFrame(summary_stats)

        print("\n=== SUMMARY STATISTICS ===")
        print(summary_df.to_string(index=False))
        summary_df.to_csv("hospital_summary_statistics3.csv", index=False)

        fig, ax = plt.subplots(figsize=(12, len(summary_df) * 0.4))
        ax.axis('off')
        table = ax.table(cellText=summary_df.values,
                         colLabels=summary_df.columns,
                         cellLoc='left',
                         loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 1.2)
        plt.title("Hospital Allocation Summary Statistics", fontsize=14, weight='bold', pad=20)
        plt.tight_layout()
        plt.savefig("hospital_summary_statistics3.png", dpi=300)
        plt.close()

    return all_assignments, rooms, patients, results

In [19]:
# Experiments
# Experiment A: Wmove, Wfair_high, Wneq (high importance)
Wmove_vals = [6, 8, 10] # Possible values for Wmove
Wfair_high_vals = [5, 7, 9] # Possible values for Wfair_high
Wneq_vals = [6, 8, 10] # Possible values for Wneq
experiment_A = list(product(Wmove_vals, Wfair_high_vals, Wneq_vals)) # Generate all combinations

# Experiment B: Wspec_mismatch, Wspec_major, Wage (medium importance)
Wspec_mismatch_vals = [4, 6, 7] # Possible values for Wspec_mismatch
Wage_vals = [4, 6, 8] # Possible values for Wage
Wspec_major_vals = [4, 6,8] # Possible values for Wspec_major
experiment_B = list(product(Wspec_mismatch_vals, Wage_vals, Wspec_major_vals)) # Generate all combinations

# Experiment C: Wfair_med, Wgen, Wpeq (equipment & preference)
Wfair_med_vals = [4, 6, 8] # Possible values for Wfair_med
Wgen_vals = [3, 5, 7] # Possible values for Wgen
Wpeq_vals = [2, 3, 5] # Possible values for Wpeq
experiment_C = list(product(Wfair_med_vals, Wgen_vals, Wpeq_vals)) # Generate all combinations

# Experiment D: Wcap, Wspec_minor, Wfair_low (lower importance/general)
Wcap_vals = [2, 3, 5] # Possible values for Wcap
Wspec_minor_vals = [1, 2, 3] # Possible values for Wspec_minor
Wfair_low_vals = [0, 1, 2] # Possible values for Wfair_low
experiment_D = list(product(Wcap_vals, Wspec_minor_vals, Wfair_low_vals)) # Generate all combinations


# Run experiments and collect results
def run_experiment(experiments, fixed_weights, label):
    results = [] # List for each combination results
    infeasible_cases = [] # List for infeasible cases details
    # Go through each combination of weights
    for combo in experiments:
        weights = fixed_weights.copy() # Start with the fixed base weights
        for i, key in enumerate([k for k in weights if weights[k] == 'vary']):
            weights[key] = combo[i]

        print(f"Running: {weights}")
        result = weights.copy() # Initialize the result dictionary
        try:
            # Solve the hospital allocation with the current weights
            _, rooms, patients, daily_results = solve_hospital_allocation(**weights)

            # If successful, record performance metrics
            result.update({
                'total_cost': sum(r['total_violation_score'] for r in daily_results), # Total violation
                'fairness_std': sum(r['fairness_violations'] for r in daily_results) / len(daily_results), # Fairness violations
                'gini': sum(r.get('gini', 0) for r in daily_results) / len(daily_results), # Gini coefficient
                'jain': sum(r.get('jain', 0) for r in daily_results) / len(daily_results), # Jain's index
                'num_patients': len(patients), # Total number of patients
                'total_rooms': len(rooms), # Total number of rooms
                'status': 'feasible' # Indicate that a feasible solution was found
            })
        except Exception as e:
            # In case of infeasible solution, record the weights and error
            print(f"Infeasible or error: {weights} → {e}")
            infeasible_cases.append({'weights': weights, 'error': str(e)})
            # Update the result dictionary
            result.update({
                'total_cost': None,
                'fairness_std': None,
                'gini': None,
                'jain': None,
                'num_patients': None,
                'total_rooms': None,
                'status': 'infeasible'
            })
        results.append(result) # Add the result to the list

    df = pd.DataFrame(results)
    df.to_csv(f"experiment_{label}.csv", index=False) # Save as CSV file
    print(f"Saved experiment {label} to experiment_{label}.csv") # Confirmation

    if infeasible_cases:
        with open(f"infeasible_{label}.json", "w") as f:
            json.dump(infeasible_cases, f, indent=2) # Save the infeasible cases list
        print(f" Saved {len(infeasible_cases)} infeasible configs to infeasible_{label}.json") # Print confirmation

    return df # Return results

# Base weights for experiments
base_weights = {
    'Wcap': 0, 'Wgen': 0, 'Wpeq': 0, 'Wneq': 0, 'Wage': 0,
    'Wspec_minor': 0, 'Wspec_major': 0, 'Wspec_mismatch': 0, 'Wfair_high': 0,
    'Wfair_med': 0, 'Wfair_low': 0,'Wmove': 0
}
# Run each experiment
# exp_A = run_experiment(experiment_A, {**base_weights, 'Wmove': 'vary', 'Wfair_high': 'vary', 'Wneq': 'vary'}, label='A')
# exp_B = run_experiment(experiment_B, {**base_weights, 'Wspec_mismatch': 'vary', 'Wage': 'vary', 'Wspec_major': 'vary'}, label='B')
# exp_C = run_experiment(experiment_C, {**base_weights, 'Wfair_med': 'vary', 'Wgen': 'vary', 'Wpeq': 'vary'}, label='C')
exp_D = run_experiment(experiment_D, {**base_weights, 'Wcap': 'vary', 'Wspec_minor': 'vary', 'Wfair_low': 'vary'}, label='D')


from google.colab import files
files.download("experiment_D.csv") # Download the results in CSV

Running: {'Wcap': 2, 'Wgen': 0, 'Wpeq': 0, 'Wneq': 0, 'Wage': 0, 'Wspec_minor': 1, 'Wspec_major': 0, 'Wspec_mismatch': 0, 'Wfair_high': 0, 'Wfair_med': 0, 'Wfair_low': 0, 'Wmove': 0}
Solving day 0...
Day 0 (initial) thresholds - Low: 0.00, High: 2.00
Day 0 solved in 0.32s — OPTIMAL
Solving day 1...
Day 1 thresholds - Low: 0.00, High: 2.00
Day 1 solved in 0.64s — OPTIMAL
Solving day 2...
Day 2 thresholds - Low: 0.00, High: 2.00
Day 2 solved in 1.09s — OPTIMAL
Solving day 3...
Day 3 thresholds - Low: 0.00, High: 2.00
Day 3 solved in 3.03s — OPTIMAL
Solving day 4...
Day 4 thresholds - Low: 0.00, High: 2.00
Day 4 solved in 4.72s — OPTIMAL
Solving day 5...
Day 5 thresholds - Low: 0.00, High: 2.00
Day 5 solved in 5.86s — OPTIMAL
Solving day 6...
Day 6 thresholds - Low: 0.00, High: 2.00
Day 6 solved in 6.42s — OPTIMAL
Solving day 7...
Day 7 thresholds - Low: 0.00, High: 2.00
Day 7 solved in 8.11s — OPTIMAL
Solving day 8...
Day 8 thresholds - Low: 0.00, High: 2.00
Day 8 solved in 8.27s — OPTIM

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>